In [2]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), '../'))

import pandas as pd
import numpy as np

import torch
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

from src import data, models

I0904 09:56:23.150138 4554018240 file_utils.py:41] PyTorch version 1.5.0 available.
I0904 09:56:23.151214 4554018240 file_utils.py:57] TensorFlow version 2.2.0-rc3 available.


In [4]:
val_df = data.load_data.load_custom_text_as_pd('../data/raw/COVID19Tweet/valid.tsv',sep='\t', header=False, \
                          text_column=['Text'],target_column=['Label'])
val_df = pd.DataFrame(val_df,copy=False)
val_df.columns = ['Id','words','labels']

In [5]:
val_df.head(5)

,Id,words,labels
0,1241728922192142336,For those saying Pakistan isn’t Italy; After 3...,UNINFORMATIVE
1,1235713405992030209,Second case DR 🇩🇴 The Canadian woman has not b...,INFORMATIVE
2,1245941302367305728,Kill Chain: the cyber war on America's electio...,UNINFORMATIVE
3,1245913002840391681,Town hosts FIRST #Virtual #TownCouncil meeting...,UNINFORMATIVE
4,1240543259299987457,Report suggested that the actual number of und...,UNINFORMATIVE


In [7]:
model_save_dir = '../models/model_withoutdrop/'

In [ ]:
def captum_text_interpreter(text, model, bpetokenizer, idx2label, max_len=80, tokenizer=None, multiclass=False):
    if type(text) == list:
        text = " ".join(text)

    d = data_utils.process_data_for_transformers(text,bpetokenizer,tokenizer,0)
    d = {
            "ids": torch.tensor([d['ids']], dtype=torch.long),
            "mask": torch.tensor([d['mask']], dtype=torch.long),
            "token_type_ids": torch.tensor([d['token_type_ids']], dtype=torch.long)
        }

    try:
        orig_tokens = [0] + bpetokenizer.encode(text).ids + [2]
        orig_tokens  = [bpetokenizer.id_to_token(j) for j in orig_tokens]
    except:
        orig_tokens = tokenizer.tokenize(text,add_special_tokens=True)

    model.eval()
    if multiclass:
    	preds_proba = torch.sigmoid(model(d["ids"],d["mask"],d["token_type_ids"])).detach().cpu().numpy()
    	preds = preds_proba.argmax(-1)
    	preds_proba = preds_proba[0][preds[0][0]]
    	predicted_class = idx2label[preds[0][0]]
    else:
    	preds_proba = torch.sigmoid(model(d["ids"],d["mask"],d["token_type_ids"])).detach().cpu().numpy()
    	preds = np.round(preds_proba)
    	preds_proba = preds_proba[0][0]
    	predicted_class = idx2label[preds[0][0]]

    lig = LayerIntegratedGradients(model, model.base_model.embeddings)
    
    reference_indices = [0] + [1]*(d["ids"].shape[1]-2) + [2]
    reference_indices = torch.tensor([reference_indices], dtype=torch.long)
    
    attributions_ig, delta = lig.attribute(inputs=d["ids"],baselines=reference_indices,additional_forward_args=(d["mask"],d["token_type_ids"]), \
                                           return_convergence_delta=True)
    
    attributions = attributions_ig.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.detach().cpu().numpy()

    visualization.visualize_text([visualization.VisualizationDataRecord(
                            word_attributions=attributions,
                            pred_prob=preds_proba,
                            pred_class=predicted_class,
                            true_class=predicted_class,
                            attr_class=predicted_class,
                            attr_score=attributions.sum(),       
                            raw_input=orig_tokens,
                            convergence_score=delta)])
